In [1]:
import os
import pandas as pd

# 경로를 os.path.join()으로 수정
data_dir = os.path.join(os.getenv('HOME'), 'aiffel/kaggle_kakr_housing/data')

# train, test 데이터 경로 설정
train_data_path = os.path.join(data_dir, 'train.csv')
test_data_path = os.path.join(data_dir, 'test.csv')

# 데이터 로드
train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

# 확인 메시지 출력
print('됐어?')

됐어?


In [2]:
train.head()

# 결측값 처리 예시
train = train.fillna(train.mean())  # 수치형 컬럼의 결측값은 평균으로 대체
test = test.fillna(test.mean())

# 범주형 컬럼 원-핫 인코딩 예시
train = pd.get_dummies(train, drop_first=True)
test = pd.get_dummies(test, drop_first=True)

# 변경 사항 확인
print(test.head())
print(test.columns)

# 훈련 데이터와 테스트 데이터 컬럼 목록 확인
train_columns = train.columns
test_columns = test.columns

print(f"훈련 데이터 컬럼: {train_columns}")
print(f"테스트 데이터 컬럼: {test_columns}")

# 훈련 데이터와 테스트 데이터에서 공통된 컬럼만 선택
common_columns = train_columns.intersection(test_columns)

# 공통 컬럼만 테스트 데이터에서 선택
test = test[common_columns]

# 이제 두 데이터의 컬럼 수가 맞게 되었습니다.

# 훈련 데이터에서 원-핫 인코딩
train = pd.get_dummies(train, drop_first=True)

# 테스트 데이터에서 동일한 인코딩을 적용
test = pd.get_dummies(test, drop_first=True)

# 훈련 데이터와 테스트 데이터의 컬럼 순서 및 개수 맞추기
test = test.reindex(columns=train.columns, fill_value=0)

/tmp/ipykernel_58/1516459514.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train = train.fillna(train.mean())  # 수치형 컬럼의 결측값은 평균으로 대체
/tmp/ipykernel_58/1516459514.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test = test.fillna(test.mean())


      id  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0  15035         3       2.25         2570      7242     2.0           0   
1  15036         4       3.00         1960      5000     1.0           0   
2  15037         4       4.50         5420    101930     1.0           0   
3  15038         3       1.00         1780      7470     1.0           0   
4  15039         3       2.50         1890      6560     2.0           0   

   view  condition  grade  ...  date_20150508T000000  date_20150509T000000  \
0     0          3      7  ...                     0                     0   
1     0          5      7  ...                     0                     0   
2     0          3     11  ...                     0                     0   
3     0          3      7  ...                     0                     0   
4     0          3      7  ...                     0                     0   

   date_20150510T000000  date_20150511T000000  date_20150512T000000  \
0  

In [38]:
import numpy as np  # numpy 임포트

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

# 모델 초기화
xgb_model = XGBRegressor(random_state=42)
rf_model = RandomForestRegressor(random_state=42)

# 모델 학습
xgb_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

# 예측
xgb_pred = xgb_model.predict(X_test)
rf_pred = rf_model.predict(X_test)

In [39]:

from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

lgbm = LGBMRegressor(random_state=42)

grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=1)

grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print(grid_search.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
{'max_depth': 5, 'n_estimators': 100}


In [40]:
from sklearn.metrics import mean_squared_error
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_true), np.expm1(y_pred)))

# 예측 및 RMSE 계산
xgb_rmse = rmse(y_test, xgb_pred)
rf_rmse = rmse(y_test, rf_pred)

print("XGBoost RMSE:", xgb_rmse)
print("RandomForest RMSE:", rf_rmse)

XGBoost RMSE: 6373.796275796151
RandomForest RMSE: 2803.976707821867


In [1]:
import os

# 이제 예측을 할 수 있습니다
final_predictions = xgb_model.predict(test)
final_predictions = np.expm1(final_predictions)  # 로그 변환을 되돌리기

# 제출할 파일 경로 설정
submission_path = os.path.join(data_dir, 'submission.csv')

# 예측 결과를 CSV 파일로 저장
result = pd.DataFrame({'id': test['id'], 'price': final_predictions})
result.to_csv(submission_path, index=False)

print(f"제출 파일이 저장되었습니다: {submission_path}")

NameError: name 'xgb_model' is not defined